# Aufgabe 2: Q-Learning Tic Tac Toe



Wir initialisieren Q(s,a) für alle möglichen Zustands- Aktions-Paare mit 0. Anfangs sind also alle möglichen Aktionen gleich wahrscheinlich.

Zur Optimierung der Laufzeit berechnen wir nicht in jedem Zustand neu alle möglichen Aktionen, sondern speichern für alle bereits getroffenen Zustände die dort möglichen Aktionen.

Die Funktionen zur visuellen Ausgabe von Feldern sowie zum Bestimmen der Aktionen sind, mit kleinen Änderungen, aus der Abgabe zu Blatt 1 kopiert.

In [3]:
# Funktionen um ein Tik Tac Toe Feld auszugeben
def print_field(field):
    """
    print the given field
    inputs:
        field - (list) or (tuple) - list of 9 ints in (0,1,2)
            0 is an empty field
    """
    print_list = convert_field(field)
    for i in range(3):
        print("-"*13)
        print("| {} | {} | {} |".format( *print_list[3*i:3*i+3] ))
    print("-"*13)

def convert_field(field):
    """
    prepare field for printing by replacing characters
    inputs:
        field - (list) - list of 9 ints in (0,1,2)
            0 is an empty field
    returns:
        (list) - list of "o", "x" and " " representing the field
    """
    print_list = []
    for elem in field:
        if elem == 1:
            print_list.append("o")
        elif elem == 2:
            print_list.append("x")
        else:
            print_list.append(" ")
    return print_list

In [14]:
# Funktionen um mögliche Aktionen sowie das Spielende zu bestimmen.
from math import sqrt
def game_ended(field, get_winner=False):
    """
    checks whether or not a ttt-game has ended
    inputs:
        field - (list) - list (must not be a tuple) with integer entries 0, 1 and 2
    ouputs:
        if get_winner is False:
            (bool) - whether or not a ttt-game has ended
        else:
            (int) - sign of the winner. 0 if it is a draw
    """
    n = int(sqrt(len(field)))
    rows = [field[n*i:n*i+n] for i in range(n)] #get all rows of the field
    columns = [field[i::n] for i in range(n)]   #get all columns of the field
    diagonals = [field[::n+1], field[n-1:n**2-2:n-1]] #get both diagonals of the field

    win_lists = rows + columns + diagonals
    
    if not get_winner:
        if [1]*n in win_lists or [2]*n in win_lists:
            return True
        return False
    else:
        if not 0 in field:
            return 0
        if [1]*n in win_lists:
            return 1
        if [2]*n in win_lists:
            return 2
        

def get_actions(field):
    """
    calculate all possible actions for the given field
    inputs:
        field - (list) - list of 9 ints in (0,1,2)
            0 is an empty field
    returns:
        (list) - list of possible actions as list indices
    """
    actions = []
    if 0 in field and not game_ended(field):
        for i, elem in enumerate(field):
            if elem == 0:
                actions.append(i)
    return actions

Wir initialisieren ein Dictionary `Q_table`, in dem die bisher probierten Zustands- Aktions- Paare mit einer Wertung gespeichert werden.

Schlüssel sind dazu Tupel (Zustand, Aktion), wobei die Aktion eine Zahl von 0 bis 8 und Zustand ein Tupel mit 9 Elementen aus {0,1,2} sind.

Außerdem initialisieren wir `action_dict` um alle in einem Zustand (als Schlüssel gegeben) möglichen Aktionen zu speichern.

_**mögliche Optimierung:**
Speichere zusätzlich den Folgezustand zu jeder Aktion._

Da Reward nur am Ende eines Spiels vergeben wird, können die Werte für Q(s,a) auch nur am Ende einer Episode aktualisiert werden und nicht während der Episode.

Erst dann kann aus den vergangenen Zuständen und gewählten Aktionen ein neuer Wert für die entsprechenden Paare berechnet werden.

Der Reward in nicht-Teriminal Zuständen ist immer 0. Tic Tac Toe ist ausreichend einfach, dass diese Festlegung nicht zu Problemen beim Training führen sollte.

In [15]:
def train_q_learning(learning_rate, discount_factor, num_episodes=1e4, reward_dict={"win":1, "loss":-1, "draw":0}):
    """
    play Tic Tac Toe [num_episodes] times to learn using Q-Learning with the given learning rate and discount_factor.
    inputs:
        learning_rate - (float) in range [0,1] - alpha
        discount_factor - (float) in range [0,1] - gamma
        num_episodes - (int) - number of episodes for training
        reward_dict - (dict) - a dictionary specifying the rewards for winning, losing and draw
            -> must have keys "win", "loss", "draw"
    returns:
        (dict) - the Q-table after training with the given parameters
    """
    Q_table = dict()
    action_dict = dict()
    exploration_rate = 1

    for n in range(num_episodes):
        # play episode
        states, actions = play_episode(Q_table, action_dict, exploration_rate)
        # evaluate results
        winner = game_ended(list(states[-1]), get_winner=True)
        states.reversed()
        actions.reversed()
        # pair with rewards for player 2 and 1 (in that order)
        if winner == 0:
            reward = (reward_dict["draw"], reward_dict["draw"])
        else:
            reward = (reward_dict["win"], reward_dict["loss"])

        for i, state, action in zip([i for i in range(9)], states, actions):
            if i == 0: # terminal State
                Q_table[(state, action)] += learning_rate * reward[0] #-Q_table[(state, action)] # ???
                next_state = state
            else:
                next_rewards = [Q_table(next_state, action) for action in action_dict(next_state)] # TODO
                Q_table[(state, action)] += learning_rate * (max(next_rewards) - Q_table[(state, action)])
                next_state = state
        

Während dem Training spielt der Algorithmus gegen sich selbst und erforscht so gleichzeitig die Zustände, wenn er selbst anfängt und wenn der Gegner anfängt.

Durch festlegen des Zeichens des Startspielers ist in jedem Zustand eindeutig, welches Zeichen als nächstes gesetzt werden muss.

Zur späteren Aktualisierung der Q-Werte werden alle Zustände und gewählten Aktionen gespeichert (in den Listen `state_history` und `action_history`).

In [ ]:
def play_episode(Q_table, action_dict, exploration_rate):
    """
    self-play an entire episode
    returns:
        (list) - state history
        (list) - action history
    
    action_dict is changed in-place
    """ 
    field = [0 for _ in range(9)]
    sign = 1
    action_history = []
    state_history = []
    while True:
        state = tuple(field)
        # get possible actions
        try:
            actions = action_dict[state]
        except KeyError:
            actions = get_actions(field)
            action_dict[state] = actions
        if actions == []:
            break # game has ended

        action = choose_action(state, actions, Q_table, exploration_rate)
        field[action] = sign
        sign = sign%2 + 1 # toggle sign between 1 and 2

        action_history.append(action)
        state_history.append(state)
    
    return state_history, action_history

Zum Wählen der Aktion wird eine $\varepsilon$-greedy Strategie genutzt.

Wird damit ausgewählt bekanntes Wissen zu nutzen, so wird eine zufällige Aktion mit maximalem Wert ausgewählt, falls es mehrere Aktionen mit maximalem Wert gibt.

In [30]:
import random
def choose_action(state, actions, Q_table, exploration_rate):
    """
    choose an action based on the possible actions, the current Q-table and the current exploration rate
    """
    r = random.random()
    if r > exploration_rate:
        # exploit knowledge
        action_values = []
        for action in actions:
            try:
                action_values.append(Q_table[(state,action)])
            except KeyError:
                action_values.append(0)
        max_value = max(action_values)
        best_actions = []
        for action, value in zip(actions, action_values):
            if value == max_value:
                best_actions.append(action)
        # return random action with maximum expected reward
        return random.choice(best_actions)

    # explore environment through random move
    return random.choice(actions)

In [50]:
field = [1,1,2,2,0,1,0,2,1]
print_field(field)
actions = get_actions(field)
choose_action(tuple(field), actions, dict(), 0.5)

-------------
| o | o | x |
-------------
| x |   | o |
-------------
|   | x | o |
-------------
[[1, 0, 1], [2, 0, 0]]


6